In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# <font color='orange'>01. load data

In [2]:
data_rootpath = r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\dev_data'

In [3]:
os.listdir(data_rootpath)

['model_dev_cb_df.csv',
 'model_dev_inner_df.csv',
 'model_valid_cb_df.csv',
 'model_valid_inner_df.csv']

In [4]:
dev_inner_df = pd.read_csv(os.path.join(data_rootpath, 'model_dev_inner_df.csv'))
valid_inner_df = pd.read_csv(os.path.join(data_rootpath, 'model_valid_inner_df.csv'))

In [5]:
# dictionary에 담기
df_dict = dict()

df_dict['dev_inner'] = dev_inner_df
df_dict['valid_inner'] = valid_inner_df

In [6]:
for key, df in df_dict.items():
    print(key, df.shape)

dev_inner (240016, 67)
valid_inner (40942, 67)


In [7]:
df_dict['dev_inner'].head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,EXECUTE_FG,DESTRUCT_FG,Y_2017_FG,Y_2018_FG,...,BAD_금융_카드_12,BAD_금융_캐피탈_12,BAD_금융_저축은행_12,BAD_대부_12,BAD_금융_6,BAD_금융_카드_6,BAD_금융_캐피탈_6,BAD_금융_저축은행_6,BAD_대부_6,BAD
0,1,201612,NaN,5.0,5.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,201612,D,5.0,2.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# 모형검증데이터 월별 나누기
df_dict['valid_inner_1908'] = valid_inner_df[valid_inner_df['신청월']==201908]
df_dict['valid_inner_1909'] = valid_inner_df[valid_inner_df['신청월']==201909]
df_dict['valid_inner_1910'] = valid_inner_df[valid_inner_df['신청월']==201910]

# <font color='orange'> 02. 주택담보대출 현황

In [10]:
seg_house_dict = dict()

for key ,df in df_dict.items():
    df = df[df['TOTAL_PRE_BAD_FG'] == 0]
    grp = df.groupby(by = ['TOTAL_HOUSE_FG'], as_index = False).agg(
    {
        'BAD' : ['count','sum']
    })
    
    grp.columns = ['house','cnt','bad_cnt']
    
    seg_house_dict[key] = grp

In [11]:
seg_house_dict['dev_inner']

,house,cnt,bad_cnt
0,0,184363,75856
1,1,14356,5295


In [12]:
# 한 엑셀파일에 여러 sheet로 저장


save_path = os.path.join(r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\SEG요약현황', '주택담보대출_요약현황_결과.xlsx')
with pd.ExcelWriter(save_path) as writer:
    for key, result in seg_house_dict.items():
        result.to_excel(writer, sheet_name = key, )

# <font color='orange'>03. SEG 현황

In [13]:
seg_dict = dict()

for key ,df in df_dict.items():
    df = df[df['TOTAL_PRE_BAD_FG'] == 0]
    df = df[df['TOTAL_HOUSE_FG'] == 0]
    grp = df.groupby(by = ['FINAL_SEG'], as_index = False).agg(
    {
        'BAD' : ['count','sum']
    })
    
    grp.columns = ['seg','cnt','bad_cnt']
    
    seg_dict[key] = grp

In [15]:
seg_dict['dev_inner']

,seg,cnt,bad_cnt
0,0,14940,7334
1,1,129813,52169
2,2,39610,16353


In [16]:
# 한 엑셀파일에 여러 sheet로 저장


save_path = os.path.join(r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\SEG요약현황', 'SEG_요약현황_결과.xlsx')
with pd.ExcelWriter(save_path) as writer:
    for key, result in seg_dict.items():
        result.to_excel(writer, sheet_name = key, )